In [16]:
# Jupyter Notebook: Data Cleaning for Ecommerce Consumer Behavior Analysis

# Cell 1: Import libraries
import pandas as pd
import numpy as np
from unidecode import unidecode

# Display all columns for inspection
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)


In [17]:
# Cell 2: Load the data
file_path = 'EcommerceConsumerBehaviorAnalysisData_COS20089_A2.csv'
df = pd.read_csv(file_path)

# Initial inspection
display(df.head())
df.info()



,Customer_ID,Age,Gender,Income_Level,Marital_Status,Education_Level,Occupation,Location,Purchase_Category,Purchase_Amount,Frequency_of_Purchase,Purchase_Channel,Brand_Loyalty,Product_Rating,Time_Spent_on_Product_Research(hours),Social_Media_Influence,Discount_Sensitivity,Return_Rate,Customer_Satisfaction,Engagement_with_Ads,Device_Used_for_Shopping,Payment_Method,Time_of_Purchase,Discount_Used,Customer_Loyalty_Program_Member,Purchase_Intent,Shipping_Preference,Time_to_Decision
0,37-611-6911,22,Female,Middle,Married,Bachelor's,Middle,Évry,Gardening & Outdoors,$333.80,4,Mixed,5,5,2.0,NaN,Somewhat Sensitive,1,7,NaN,Tablet,Credit Card,3/1/2024,True,False,Need-based,No Preference,2
1,29-392-9296,49,Male,High,Married,High School,High,Huocheng,Food & Beverages,$222.22,11,In-Store,3,1,2.0,Medium,Not Sensitive,1,5,High,Tablet,PayPal,4/16/2024,True,False,Wants-based,Standard,6
2,84-649-5117,24,Female,Middle,Single,Master's,High,Huzhen,Office Supplies,$426.22,2,Mixed,5,5,0.3,Low,Not Sensitive,1,7,Low,Smartphone,Debit Card,3/15/2024,True,True,Impulsive,No Preference,3
3,48-980-6078,29,Female,Middle,Single,Master's,Middle,Wiwilí,Home Appliances,$101.31,6,Mixed,3,1,1.0,High,Somewhat Sensitive,0,1,NaN,Smartphone,Other,10/4/2024,True,True,Need-based,Express,10
4,91-170-9072,33,Female,Middle,Widowed,High School,Middle,Nara,Furniture,$211.70,6,Mixed,3,4,0.0,Medium,Not Sensitive,2,10,NaN,Smartphone,Debit Card,1/30/2024,False,False,Wants-based,No Preference,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Customer_ID                            1000 non-null   object 
 1   Age                                    1000 non-null   int64  
 2   Gender                                 1000 non-null   object 
 3   Income_Level                           1000 non-null   object 
 4   Marital_Status                         1000 non-null   object 
 5   Education_Level                        1000 non-null   object 
 6   Occupation                             1000 non-null   object 
 7   Location                               1000 non-null   object 
 8   Purchase_Category                      1000 non-null   object 
 9   Purchase_Amount                        1000 non-null   object 
 10  Frequency_of_Purchase                  1000 non-null   int64  
 11  Purch

In [18]:
# Cell 3: Clean the Location column
#  - Strip leading/trailing whitespace
#  - Normalize text (remove accents, unify case)
#  - Keep 'None' entries unchanged
def clean_location(loc):
    if pd.isna(loc) or loc == 'None':
        return loc  # preserve NaN or literal 'None'
    loc_str = str(loc).strip()
    loc_norm = unidecode(loc_str).title()
    return loc_norm

# Apply cleaning
df['Location'] = df['Location'].apply(clean_location)

# Verify uniqueness
display(df['Location'].unique())



array(['Evry', 'Huocheng', 'Huzhen', 'Wiwili', 'Nara', 'Boro Utara',
       'Liren', 'Taocheng', 'Grabo', 'Pryamitsyno', 'Punta Gorda', 'Gyor',
       'Gaosheng', 'Rokytne', 'Alameda', 'Dhankuta', 'Jindong', 'Ngeru',
       'White Rock', 'Monastyrshchina', 'Pangkalanbunut', 'Lederaba',
       'Lugo', 'Sao Roque', 'Bani Khaddash', 'Montpellier', 'Nesovice',
       'Pasirhuni', 'Glendale', 'Tala', 'Marseille', 'Nasielsk',
       'Paris 06', 'Bronnitsy', 'Inayauan', 'Qianjin', 'Bagahanlad',
       'Rzewnie', 'Strabychovo', 'Hongqi', 'Oeoh', "Topol'Noye",
       'Kovarov', 'Majdal Bani Fadil', 'Delta Del Tigre',
       'Santa Cruz La Laguna', 'Manchester', 'Norrkoping', 'Sosno',
       'Pella', 'Namur', 'Benevides', 'Qinghaihu', 'Alfeizerao',
       'Prostredni Becva', 'Oeiras', 'Kythnos', 'Rizal', 'Fullerton',
       'Huaidao', 'Bilice', 'Isla Verde', 'Hunkuyi', 'Shiree',
       'Strezhevoy', 'Longxing', 'Jesus Menendez', 'Hongguang', 'Pilaya',
       'Camp Ithier', 'Holboo', 'Luxi', 'Ost

In [19]:
# Cell 4: Preserve Purchase_Amount as-is (keep currency symbols and formatting)
print('Purchase_Amount dtype before:', df['Purchase_Amount'].dtype)



Purchase_Amount dtype before: object


In [26]:
# Cell 5: Convert Time_of_Purchase to datetime
#  - Preserve original strings in 'Time_of_Purchase_str'
#  - For entries with 'None', keep literal 'None' in cleaned column

df['Time_of_Purchase_str'] = df['Time_of_Purchase']
df['Time_of_Purchase'] = pd.to_datetime(
    df['Time_of_Purchase_str'], infer_datetime_format=True, errors='coerce'
)
# Restore 'None' for those entries
df.loc[df['Time_of_Purchase_str'] == 'None', 'Time_of_Purchase'] = 'None'

# Verify conversion
display(df[['Time_of_Purchase_str', 'Time_of_Purchase']].head())



C:\Users\USER\AppData\Local\Temp\ipykernel_19668\2091048217.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Time_of_Purchase'] = pd.to_datetime(
C:\Users\USER\AppData\Local\Temp\ipykernel_19668\2091048217.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.loc[df['Time_of_Purchase_str'] == 'None', 'Time_of_Purchase'] = 'None'


,Time_of_Purchase_str,Time_of_Purchase
0,2024-03-01,2024-03-01 00:00:00
1,2024-04-16,2024-04-16 00:00:00
2,2024-03-15,2024-03-15 00:00:00
3,2024-10-04,2024-10-04 00:00:00
4,2024-01-30,2024-01-30 00:00:00


In [28]:
# Cell 6: Define safe numeric converters to preserve 'None'
def safe_to_float(x):
    if pd.isna(x) or x == 'None':
        return x
    try:
        return float(x)
    except Exception:
        return np.nan

# Convert research time
df['Time_Spent_on_Product_Research(hours)'] = df['Time_Spent_on_Product_Research(hours)'].apply(safe_to_float)

# Convert other integer-like columns safely
int_cols = ['Age', 'Product_Rating', 'Return_Rate', 'Customer_Satisfaction', 'Time_to_Decision']
for col in int_cols:
    def safe_to_int(x):
        if pd.isna(x) or x == 'None':
            return x
        try:
            return int(float(x))
        except Exception:
            return np.nan
    df[col] = df[col].apply(safe_to_int)

# Leave Social_Media_Influence and Engagement_with_Ads untouched to prevent data loss
display(df[['Social_Media_Influence', 'Engagement_with_Ads']].head())



,Social_Media_Influence,Engagement_with_Ads
0,None,None
1,Medium,High
2,Low,Low
3,High,None
4,Medium,None


In [29]:
# Cell 7: Final overview
display(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Customer_ID                            1000 non-null   object        
 1   Age                                    1000 non-null   int64         
 2   Gender                                 1000 non-null   object        
 3   Income_Level                           1000 non-null   object        
 4   Marital_Status                         1000 non-null   object        
 5   Education_Level                        1000 non-null   object        
 6   Occupation                             1000 non-null   object        
 7   Location                               1000 non-null   object        
 8   Purchase_Category                      1000 non-null   object        
 9   Purchase_Amount                        1000 non-null   object   

None

In [31]:
# Cell 8: Replace all remaining NaN values with literal 'None'
# This ensures no blank entries remain

df = df.fillna('None')


In [33]:
# Cell 9: Save cleaned dataset
df.to_csv('EcommerceConsumerBehavior_Cleaned.csv', index=False)
print("Cleaned dataset saved to EcommerceConsumerBehavior_Cleaned.csv")
display(df.info())

# Save cleaned data
df.to_csv('EcommerceConsumerBehavior_Cleaned.csv', index=False)
print("Cleaned dataset saved to EcommerceConsumerBehavior_Cleaned.csv")

Cleaned dataset saved to EcommerceConsumerBehavior_Cleaned.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Customer_ID                            1000 non-null   object        
 1   Age                                    1000 non-null   int64         
 2   Gender                                 1000 non-null   object        
 3   Income_Level                           1000 non-null   object        
 4   Marital_Status                         1000 non-null   object        
 5   Education_Level                        1000 non-null   object        
 6   Occupation                             1000 non-null   object        
 7   Location                               1000 non-null   object        
 8   Purchase_Category                      1000 non-null   object        
 9   P

None

Cleaned dataset saved to EcommerceConsumerBehavior_Cleaned.csv
